In [10]:
# installing packages
!pip install -q langchain
!pip install -q gradio
!pip install -q accelerate
!pip install -q sentence-transformers metapub
!pip install -q transformers accelerate bitsandbytes
!pip install -q transformers trl py7zr optimum accelerate bitsandbytes
!pip install -q langchain-huggingface
!pip install -q faiss-cpu
!pip install -q llama-index
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.9 MB/s eta 0:00:00


In [52]:
# importing libraries
import pandas as pd
import gradio as gr
import numpy as np
import torch
import os
from langchain.memory import  ConversationSummaryBufferMemory
from langchain.chains import LLMChain, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModel, pipeline, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [ ]:
# First importing models and embeddings and memory
model_id = "MehdiHosseiniMoghadam/AVA-Llama-3-V2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization_config = BitsAndBytesConfig(
    low_cpu_mem_usage=True,
    load_in_8bit=True,
    torch_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quantization_config)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    config={
        "max_length": 512,
        "temperature": 0.2,
        "top_k": 10,
        "top_p": 0.3,
        "do_sample": True,
    }
)


llm = HuggingFacePipeline(pipeline=pipe)

model_name = "HooshvareLab/bert-base-parsbert-uncased"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unused kwargs: ['low_cpu_mem_usage', 'torch_dtype']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
model_name = "HooshvareLab/bert-base-parsbert-uncased"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

In [17]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [32]:
# RAG
# reading documents
documents = TextLoader("/content/drive/MyDrive/Dataset/Hormozgan_Dataset/uni_dataset.csv").load()
documents

[Document(page_content='Texts\n"\n\n \n  یฮعاৎ هૡ\u0cf6য \nتقويم آموزشی دوره تابستان ٠٠۴١-٩٩٣١   \n \n \nماه هفته شنبه يكشنبه دوشنبه سه شنبه چهارشنبه پنجشنبه جمعه \nتير \n    ثبت نام از  ٤٢/٤٠/٠٠٤١ الي ٦٢/٤٠/٠٠٤١ ، شروع كﻼس ها از ٧٢/٤٠/٠٠٤١  \n ٩١ ٠٢ ١٢ ٢٢ ٣٢ ٤٢ ٥٢ \n \n١ ٦٢  ٧٢  ٨٢ ٩٢ ٠٣ ١٣ ١ \nمرداد ٢ ٢ ٣ ٤ ٥ ٦ ٧ ٨ \n٣ ٩ ٠١ ١١ ٢١ ٣١ ٤١ ٥١ \n٤ ٦١ ٧١ ٨١  ٩١ ٠٢ ١٢ ٢٢ \n٥ ٣٢ ٤٢ ٥٢  ٦٢  ٧٢ ٨٢ ٩٢ \n٦ ٠٣ ١٣ ١  ٢ \n  ٣ ٤ ٥ \nشهريور  امتحانات پايان دوره تابستان از ٨٠/٦٠/٠٠٤١ الي ٠١/٦٠/٠٠٤١ \n٧ ٦ ٧ ٨ ٩ ٠١ ١١ ٢١ \n ٣١ \nقفل نمرات ٤١ ٥١ ٦١ ٧١ ٨١ ٩١ \n]\nIndex: []"\n"\n \n))ﻓﺮآﯾﻨﺪ ﺗﻬﯿﻪ و ﺗﻌﯿﯿﻦ  اﻋﺘﺒﺎر ﺳﺮﻓﺼﻞ دروس((  \n  \n  \n- ﭘﺲ از ﺗﺼﻮﯾﺐ راه اﻧﺪازي رﺷﺘﻪ ﺟﺪﯾﺪ، ﮔﺮوه آﻣﻮزﺷﯽ ﻣﻮﻇﻒ اﺳﺖ درﺧﺼﻮص \nﺗﻬﯿﻪ ﺳﺮﻓﺼﻞ دروس ﺗﺎ ﻗﺒﻞ از ﺷﺮوع ﻧﯿﻤﺴﺎل و ﻫﻤﭽ ﻨﯿﻦ در ﺻﻮرت ﺗﻐﯿﯿﺮ  \nﺳﺮﻓﺼﻞ ﻧﺴﺒﺖ ﺑﻪ ﺗﻬﯿﻪ و اﺟﺮاي ﺑﻤﻮﻗﻊ ﺳﺮﻓﺼﻞ ﺟﺪﯾﺪ اﻗﺪام ﻧﻤﺎﯾﺪ.  \n- ﺿﺮوري اﺳﺖ ﺳﺮﻓﺼﻞ دروس ﻣﻤﻬﻮر ﺑﻪ ﻣﻬﺮ وزارت ﻣﺘﺒﻮع ﯾﺎ ﻫﯿﺎت ﻣﻤﯿﺰه و ﯾﺎ  \nﻣﻌﺎوﻧﺖ آﻣﻮزﺷﯽ داﻧﺸﮕﺎه ﺑﻮده و ﺗﺎرﯾﺦ اﺟﺮاي آن ﻣﺸﺨﺺ ﺑﺎﺷﺪ.  \n- ﺑﺎﯾﺴﺘﻪ اﺳﺖ ارﺳﺎل ﺳﺮﻓﺼﻞ دروس ﻫﻤﺮاه ﺑﺎ ﻧﺎﻣﻪ رﺳﻤﯽ ﻣﻘﺎم و ﯾﺎ 

In [33]:
# retrievers and Vectorstore
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=25)
documents = text_splitter.split_documents(documents)
db = FAISS.from_documents(documents, hf)

In [50]:
# Example query
query = "درس ریاضی 2 پیش نیازش چه درسی می باشد؟"

# Run the RAG chain
docs = db.similarity_search(query, k=2)
print(docs[1].page_content)
# vector
embedding_vector = hf.embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[1].page_content)

IndexError: list index out of range

In [53]:

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(db, question_answer_chain)

chain.invoke({"input": query})

NameError: name 'llm' is not defined

In [ ]:
# prompt example

user_prompt = """
سلام من یک برنامه هفتگی می خواهم که شرایط زیر رعایت بشود.
و لطفا توجه داشته باشید که من برخی از دروس را گذراندم که اینها هستند:
معادله دیفرانسیل: 3 واحد - پیش نیاز: حساب دیفرانسیل و انتگرال 1
ریاضی گسسته: 3 واحد - پیش نیاز: حساب دیفرانسیل و انتگرال 1
حساب دیفرانسیل و انتگرال 1: 3 واحد
حساب دیفرانسیل و انتگرال 2: 3 واحد - پیش نیاز: حساب و دیفرانسیل و انتگرال 1
فیزیک 1: 3 واحد
فیزیک 2: 3 واحد - پیش نیاز: فیزیک 1
برنامه نویسی و مبانی کامپیوتر: 3 واحد
برنامه نویسی پیشرفته: 3 واحد - پیش نیاز: برنامه نویسی و مبانی کامپیوتر
آزمایشگاه فیزیک 2: 1 واحد
آمار و احتمالات : 3 واحد - پیش نیاز : حساب دیفرانسیل و انتگرال 2
مدار منطقی: 3 واحد -  هم نیاز: ریاضی گسسته
ساختمان داده: 3 واحد - پیش نیاز: ریاضی گسسته، برنامه نویسی پیشرفته
و من این دروس را در ترم دارم:
معماری کامپیوتر: 3 واحد - پیش نیاز: مدار منطقی
ریاضی مهندسی 3 واحد - پیش نیاز : حساب دیفرانسیل و انتگرال  2 و معادله دیفرانسیل
هوش مصنوعی و سیستم های خبره: 3 واحد - پیش نیاز: ساختمان داده
نظریه زبان و ماشین: 3 واحد - پیش نیاز: ساختمان داده
پایگاه داده: 3 واحد - پیش نیاز: ساختمان داده
طراحی و تحلیل کامپیوتر: 3 واحد - پیش نیاز: برنامه نویسی پیشرفته
من از شما می خواهم چند درس را برای ترم بعد انتخاب کنید که باید آنها را انتخاب کنم و آن را پاس نکردم یا این ترم ندارم و می خواهم فقط 18 واحد برای ترم بعدی داشته باشم، یک برنامه هفتگی کامل برای ترم بعدی ایجاد کنم که مناسب باشد. لطفا یک جدول برای آن درست کنید
"""

In [56]:
# Define your prompt templates

classification_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
    You are an intelligent assistant. Given the input text, classify it into one of the following categories:
    - Translating_persian
    - generating_code
    - create_course_table
    - Education_question
    - Q/A-question

    Input: {text}
    Category:
    """
)

Q_A_prompt = """
You are a Persian and answer in Persian/Farsi.
You are a helpful assistant. Your task is to answer user questions using only the provided context. Follow these steps:

1. Carefully read and understand the given context.
2. Think through the information step by step to formulate your answer.
3. If the user question cannot be answered based on the provided context, inform the user that you do not have the necessary information.
4. Do not use any external information or knowledge beyond the given context to answer the questions.
5. Provide a detailed and thorough answer to the question.

{context}

{history}

Question: {question}
Helpful Answer:

Use the structure and instructions above to create a consistent and helpful Q&A system.
"""
table_prompt = """
You are a Persian and answer in Persian/Farsi.
You are a helpful assistant. Your task is to create a detailed table of daily schedules for courses using the provided context. Follow these steps:

1. Carefully read and understand the given context, which includes course names, units, timings, prerequisite, co-requisite, the day that user do not want to get.
2. Organize the information into a clear and concise table format.
3. Ensure the table includes columns for the day of the week, course name, start time, end time, and location (if provided).
4. If any information is missing or unclear, make a note of it, but do not use any external information to fill in the gaps.
5. Present the table in a way that is easy to read and understand.
6. From 12:00 AM till 14:00 is lunch time.
7. The day is started from Saturday till Thursday.
8. Day starts at 8 and ends at 18.
9. Pay attention to units of each courses, 3 units means in a week this course should be held twice a week and 2 units and 1 units just once a week.
10. in some hours if there is no constraint on that hours you can just randomly fill it blank.

{context}

{history}

Please write all courses here:
1- Calculus 1 : 3 units
2- Physics 2 : 3 units - prerequisite: Physics 1
3- Computer Fundamental : 3 units
4- Compiler Design : 3 units - prerequisite: Formal Language and Automata
5- Art 1 : 2 units
6- Data Structure : 3 units : prerequisite: Advanced programming - Discrete mathematical

This Table just an example just learn the pattern form it not write the same in answer:
| Day        |    8:00 - 10:00     |    10:00 - 12: 00   |    12:00 - 14:00    |    14:00 - 16:00    |    16:00 - 18:00    |
|------------|---------------------|---------------------|---------------------|---------------------|---------------------|
| Saturday   | Calculus 1          |                     | Lunch Time          | Physics2            |                     |
| Monday     | Physics 2           | Calculus 1          | Lunch Time          |                     |                     |
| Tuesday    | Computer Fundamental| Data Structure      | Lunch Time          |                     | Art 1               |
| Wednesday  | Compiler Design     |                     | Lunch Time          |                     |                     |
| Thursday   | Art 1               |                     | Lunch Time          |                     |                     |
| Thursday   | Data Structure      |                     | Lunch Time          | Compiler Design     |                     |

Use the structure and instructions above to create a consistent and helpful table of daily schedules for courses.
"""

answering_edu_prompt = """
You are a Persian and answer in Persian/Farsi.
You are a knowledgeable assistant. Your task is to answer educational questions using the provided information. Follow these steps:

1. Carefully read and understand the given context, which includes the retrieved documents and relevant information.
2. Think through the information step by step to formulate your answer.
3. Use only the provided information to answer the questions. Do not use any external information or personal knowledge.
4. If the user question cannot be answered based on the provided information, inform the user that you do not have the necessary information.
5. Provide a detailed and thorough answer to the question.

{context}

{history}

{question}

Answer:

Use the structure and instructions above to create a consistent and helpful educational Q&A system.
"""

translating_prompt = """
You are a Persian and answer in Persian/Farsi.
You are a skilled translator. Your task is to translate the provided English text into Persian. Follow these steps:

1. Carefully read and understand the given English text.
2. Translate the text into accurate and grammatically correct Persian.
3. Preserve the original meaning and tone of the text as much as possible.
4. Ensure that the translation is clear and easy to understand.

{history}

{question}

Example:
English Text: "The quick brown fox jumps over the lazy dog."

Persian Translation: "روباه قهوه‌ای سریع از روی سگ تنبل می‌پرد."

Use the structure and instructions above to create a consistent and accurate translation.
"""

writing_code_prompt = """
You are a Persian and for explanation answer in Persian and for code answer in English.
"You are a proficient programmer. Your task is to write code based on the provided requirements. Follow these steps:

1. Carefully read and understand the given requirements.
2. Write clear, efficient, and well-documented code to meet the requirements.
3. Ensure the code is free of errors and follows best practices.
4. If any requirements are unclear, make reasonable assumptions and document them in the code.
5. Include comments to explain the logic and functionality of the code.

Example Requirements:
Write a Python function that takes a list of numbers and returns the sum of the even numbers in the list.

{history}

{question}

Example Code:
def sum_of_even_numbers(numbers):
    \"""
    This function takes a list of numbers and returns the sum of the even numbers in the list.

    :param numbers: List of integers
    :return: Sum of even integers in the list
    \"""
    sum_even = 0
    for num in numbers:
        if num % 2 == 0:
            sum_even += num
    return sum_even

# Example usage:
example_list = [1, 2, 3, 4, 5, 6]
print(sum_of_even_numbers(example_list))  # Output: 12

Use the structure and instructions above to write code that meets the given requirements.
"""

In [59]:
# Define your prompt templates
prompt_templates = [
    PromptTemplate(
        id="Q/A-question",
        template=Q_A_prompt,
        input_variables=["context", "history", "question"],
        is_system=True,
        weight=0.9
    ),

    PromptTemplate(
        id="create_course_table",
        template=table_prompt,
        input_variables=["context", "history"],
        is_system=True,
        weight=1.0
    ),

    PromptTemplate(
        id="Translating_persian",
        template=translating_prompt,
        input_variables=["history", "question"],
        is_system=True,
        weight=0.8
    ),

    PromptTemplate(
        id="Education_question",
        template=answering_edu_prompt,
        input_variables=["context", "history", "question"],
        is_system=True,
        weight=1.0
    ),

    PromptTemplate(
        id="generating_code",
        template=writing_code_prompt,
        input_variables=["history", "question"],
        is_system=True,
        weight=0.8
    ),
]




In [61]:
# writing LLMChain for each prompt (creating chian)
memory_prompt = PromptTemplate(input_variables=["input_text"], template="{input_text}")
memory_llm = LLMChain(llm=llm, prompt=memory_prompt)
memory = ConversationSummaryBufferMemory(llm=memory_llm, max_token_limit=100)

NameError: name 'llm' is not defined

In [62]:
print(prompt_templates[1])

input_variables=['context', 'history'] template='\nYou are a Persian and answer in Persian/Farsi.\nYou are a helpful assistant. Your task is to create a detailed table of daily schedules for courses using the provided context. Follow these steps:\n\n1. Carefully read and understand the given context, which includes course names, units, timings, prerequisite, co-requisite, the day that user do not want to get.\n2. Organize the information into a clear and concise table format.\n3. Ensure the table includes columns for the day of the week, course name, start time, end time, and location (if provided).\n4. If any information is missing or unclear, make a note of it, but do not use any external information to fill in the gaps.\n5. Present the table in a way that is easy to read and understand.\n6. From 12:00 AM till 14:00 is lunch time.\n7. The day is started from Saturday till Thursday.\n8. Day starts at 8 and ends at 18.\n9. Pay attention to units of each courses, 3 units means in a week

In [63]:
# prompt classifying

classification_chain = LLMChain(llm=llm, prompt=classification_prompt)

def classify_input_with_llm(text:str):
  classification_input = {"text": text}
  result = classification_chain.run(classification_input)
  return result.strip().lower()


def process_input(question:str):
  task_type = classify_input_with_llm(question)
  history = memory.buffer

  if task_type == "Q/A-question":
    # (question_type, Rag:bool, history:bool)
    # (task_type, True, True)
    response = str(query_engine.query(question))
    qa_input = {"question": question, "context": response, "history": history}
    prompt_template = next(pt for pt in prompt_templates if pt.id == task_type)
    prompt = prompt_template.format(**qa_input)
    return prompt

  elif task_type == "create_course_table":
    # (question_type, Rag:bool, history:bool)
    # (task_type, True, True)
    response = str(query_engine.query(question))
    table_input = {"question": question, "context": response, "history": history}
    prompt_template = next(pt for pt in prompt_templates if pt.id == task_type)
    prompt = prompt_template.format(**table_input)
    return prompt

  elif task_type == "Translating_persian":
    # (question_type, Rag:bool, history:bool)
    # (task_type, False, True)
    translate_input = {"question": question, "history": history}
    prompt_template = next(pt for pt in prompt_templates if pt.id == task_type)
    prompt = prompt_template.format(**translate_input)
    return prompt

  elif task_type == "Education_question":
    # (question_type, Rag:bool, history:bool)
    # (task_type, True, True)
    response = str(query_engine.query(question))
    edu_input = {"question": question, "history": history}
    prompt_template = next(pt for pt in prompt_templates if pt.id == task_type)
    prompt = prompt_template.format(**edu_input)
    return prompt

  elif task_type == "generating_code":
    # (question_type, Rag:bool, history:bool)
    # (task_type, False, True)
    code_input = {"question": question, "history": history}
    prompt_template = next(pt for pt in prompt_templates if pt.id == task_type)
    prompt = prompt_template.format(**code_input)
    return prompt

  else:
    return "Unknown task type. Please provide a valid input."


NameError: name 'llm' is not defined

In [ ]:
# gradio and chating

def chat(query):

    prompt = process_input(query)
    history = memory.buffer
    system_prompt = (
    "You are educational manager for educational affairs and a student advisor to help students and a teacher to learn coding and information assistant for answer about university question."
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use four sentence maximum and keep the answer concise. "
    "Context: {context}"
    )
    prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        ("assistant", "{history}")
    ])
    response = llm.chat(prompt)
    memory.add_message("user", query)
    memory.add_message("assistant", response["content"])

    return response["content"]


interface = gr.Interface(
    fn=chat,
    inputs=gr.Textbox(lines=10, placeholder="Enter your prompt"),
    outputs="text"
)
interface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://15fd370c5cbcebdcf1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 532, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1923, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1509, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    r

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://15fd370c5cbcebdcf1.gradio.live
